In [ ]:
import pandas as pd
import numpy as np
import ast
from collections import Counter
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Load the L1_Master_sheet_92185
master = pd.read_csv('L1_Master_sheet_92185.csv')
print(f"Data shape: {master.shape}")
print(f"Unique commodities: {master['Commodity'].nunique()}")
print(f"\nColumns: {master.columns.tolist()}")
master.head()


Data shape: (92185, 7)
Unique commodities: 36

Columns: ['TWC_No', 'Commodity', 'Problem', 'Condition', 'Failure Mode', 'Symptom Mode', 'Fix Mode']


,TWC_No,Commodity,Problem,Condition,Failure Mode,Symptom Mode,Fix Mode
0,4081194,87106,Stuck/Seized Components,AC/Noise/System Issues,['Stuck - Actuator'],['Noise - Dashboard'],['Replaced - Servo Damper']
1,3908966,87106,Stuck/Seized Components,AC System Inoperative,['Stuck - Servo Motor'],['Inoperative - Air Conditioning System'],['Replaced - Servo Motor']
2,3960333,87106,Servo/Motor/Actuator Issue,AC Temperature & Airflow Issues,"['Malfunction - Navigation', 'Malfunction - An...","['Blowing Hot Air - Air Conditioning System', ...",['Performed Servo Motor Initialization - No Co...
3,3997708,87106,HVAC/Servo Malfunctions,AC & Temperature Control Issues,['No Condition Mentioned - No Component Mentio...,['Blowing Hot Air - Air Conditioning System'],['Replaced - Radiator Servo Damper']
4,4042723,87106,Open/Short Circuit/Electrical Issue,AC & Temperature Control Issues,['Open Circuit - Servo Drive'],['Blowing Hot Air - Passenger Side AC'],"['Removed - Servo Motor', 'Replaced - Servo Mo..."


In [16]:
# Parse Failure Mode and Symptom Mode from string representation to lists
def parse_list_string(s):
    """Parse string representation of list to actual list"""
    if pd.isna(s) or s == '':
        return []
    try:
        # Try to evaluate as Python literal
        parsed = ast.literal_eval(str(s))
        if isinstance(parsed, list):
            return [str(x).strip() for x in parsed if str(x).strip()]
        else:
            return [str(parsed).strip()] if str(parsed).strip() else []
    except:
        # If parsing fails, treat as single string
        return [str(s).strip()] if str(s).strip() else []

# Apply parsing
master['Failure Mode Parsed'] = master['Failure Mode'].apply(parse_list_string)
master['Symptom Mode Parsed'] = master['Symptom Mode'].apply(parse_list_string)

# Create text representations for embedding
master['Failure Mode Text'] = master['Failure Mode Parsed'].apply(
    lambda x: ' '.join(x) if x else 'No Failure Mode'
)
master['Symptom Mode Text'] = master['Symptom Mode Parsed'].apply(
    lambda x: ' '.join(x) if x else 'No Symptom Mode'
)

print("Sample parsed data:")
print(master[['Commodity', 'Failure Mode Text', 'Symptom Mode Text', 'Problem', 'Condition']].head(3))


Sample parsed data:
  Commodity                                  Failure Mode Text  \
0     87106                                   Stuck - Actuator   
1     87106                                Stuck - Servo Motor   
2     87106  Malfunction - Navigation Malfunction - Anti Ma...   

                                   Symptom Mode Text  \
0                                  Noise - Dashboard   
1              Inoperative - Air Conditioning System   
2  Blowing Hot Air - Air Conditioning System Inop...   

                      Problem                        Condition  
0     Stuck/Seized Components           AC/Noise/System Issues  
1     Stuck/Seized Components            AC System Inoperative  
2  Servo/Motor/Actuator Issue  AC Temperature & Airflow Issues  


In [ ]:
# Create commodity-specific category mappings for constraint enforcement
commodity_problem_map = {}
commodity_condition_map = {}

for commodity in master['Commodity'].unique():
    comm_df = master[master['Commodity'] == commodity]
    # Filter out NaN values and convert to strings, then sort
    problems = [str(p) for p in comm_df['Problem'].unique() if pd.notna(p) and str(p).strip() != '']
    conditions = [str(c) for c in comm_df['Condition'].unique() if pd.notna(c) and str(c).strip() != '']
    commodity_problem_map[commodity] = sorted(problems)
    commodity_condition_map[commodity] = sorted(conditions)

print(f"Created mappings for {len(commodity_problem_map)} commodities")
# Use string key since commodities are stored as strings
example_commodity = '87106'
print(f"\nExample - Commodity {example_commodity}:")
print(f"  Problems: {len(commodity_problem_map[example_commodity])} categories")
print(f"  Conditions: {len(commodity_condition_map[example_commodity])} categories")
print(f"\nSample Problems: {commodity_problem_map[example_commodity][:3]}")
print(f"Sample Conditions: {commodity_condition_map[example_commodity][:3]}")


Created mappings for 36 commodities

Example - Commodity 87106:
  Problems: 11 categories
  Conditions: 14 categories

Sample Problems: ['Electrical/Programming/Communication Issue', 'Faulty Door & HVAC Components', 'HVAC/Servo Malfunctions']
Sample Conditions: ['AC & Climate Control Issues', 'AC & Heating Inoperative', 'AC & Temperature Control Issues']


In [ ]:
# Prepare features: Combine Commodity with Failure/Symptom Mode for context
master['Text_Problem'] = master.apply(
    lambda row: f"Commodity_{row['Commodity']} Failure: {row['Failure Mode Text']}", 
    axis=1
)
master['Text_Condition'] = master.apply(
    lambda row: f"Commodity_{row['Commodity']} Symptom: {row['Symptom Mode Text']}", 
    axis=1
)

# Filter out rows with missing target values and ensure they're strings
df_problem = master[
    master['Problem'].notna() & 
    (master['Problem'] != '') & 
    (master['Problem'].astype(str).str.strip() != '')
].copy()
df_condition = master[
    master['Condition'].notna() & 
    (master['Condition'] != '') & 
    (master['Condition'].astype(str).str.strip() != '')
].copy()

# Convert to strings to avoid type issues
df_problem['Problem'] = df_problem['Problem'].astype(str)
df_condition['Condition'] = df_condition['Condition'].astype(str)

print(f"Problem dataset: {len(df_problem)} rows")
print(f"Condition dataset: {len(df_condition)} rows")
print(f"\nSample Problem text: {df_problem['Text_Problem'].iloc[0]}")
print(f"Sample Condition text: {df_condition['Text_Condition'].iloc[0]}")


Problem dataset: 92013 rows
Condition dataset: 92012 rows

Sample Problem text: Commodity_87106 Failure: Stuck - Actuator
Sample Condition text: Commodity_87106 Symptom: Noise - Dashboard


In [ ]:
# Load sentence transformer model for embeddings
print("Loading sentence transformer model...")
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
print("Model loaded successfully!")

# Generate embeddings for Problem classification
print("\nGenerating embeddings for Problem classification...")
X_problem_text = df_problem['Text_Problem'].tolist()
X_problem_embeddings = embedding_model.encode(
    X_problem_text, 
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True
)
y_problem = df_problem['Problem'].values

print(f"Problem embeddings shape: {X_problem_embeddings.shape}")
print(f"Problem labels shape: {y_problem.shape}")


Loading sentence transformer model...
Model loaded successfully!

Generating embeddings for Problem classification...


Batches: 100%|██████████| 2876/2876 [00:38<00:00, 74.15it/s]


Problem embeddings shape: (92013, 384)
Problem labels shape: (92013,)


In [20]:
# Generate embeddings for Condition classification
print("Generating embeddings for Condition classification...")
X_condition_text = df_condition['Text_Condition'].tolist()
X_condition_embeddings = embedding_model.encode(
    X_condition_text,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True
)
y_condition = df_condition['Condition'].values

print(f"Condition embeddings shape: {X_condition_embeddings.shape}")
print(f"Condition labels shape: {y_condition.shape}")


Generating embeddings for Condition classification...


Batches: 100%|██████████| 2876/2876 [00:42<00:00, 67.01it/s]


Condition embeddings shape: (92012, 384)
Condition labels shape: (92012,)


In [ ]:
# Split data for Problem classification
# Store indices to track which rows are in test set
indices_prob = np.arange(len(df_problem))

# Check if stratification is possible (all classes need at least 2 samples)
class_counts = Counter(y_problem)
min_class_count = min(class_counts.values())
can_stratify = min_class_count >= 2

if can_stratify:
    print("Using stratified split for Problem classification...")
    X_train_prob, X_test_prob, y_train_prob, y_test_prob, idx_train_prob, idx_test_prob = train_test_split(
        X_problem_embeddings, 
        y_problem,
        indices_prob,
        test_size=0.2, 
        random_state=42,
        stratify=y_problem
    )
else:
    print(f"Warning: Some Problem classes have only 1 sample. Using non-stratified split.")
    print(f"Classes with < 2 samples: {sum(1 for count in class_counts.values() if count < 2)}")
    X_train_prob, X_test_prob, y_train_prob, y_test_prob, idx_train_prob, idx_test_prob = train_test_split(
        X_problem_embeddings, 
        y_problem,
        indices_prob,
        test_size=0.2, 
        random_state=42
    )

print(f"Problem Training set: {X_train_prob.shape[0]} samples")
print(f"Problem Test set: {X_test_prob.shape[0]} samples")
print(f"Unique Problem classes: {len(np.unique(y_problem))}")


Classes with < 2 samples: 9
Problem Training set: 73610 samples
Problem Test set: 18403 samples
Unique Problem classes: 183


In [ ]:
# Split data for Condition classification
# Store indices to track which rows are in test set
indices_cond = np.arange(len(df_condition))

# Check if stratification is possible (all classes need at least 2 samples)
class_counts_cond = Counter(y_condition)
min_class_count_cond = min(class_counts_cond.values())
can_stratify_cond = min_class_count_cond >= 2

if can_stratify_cond:
    print("Using stratified split for Condition classification...")
    X_train_cond, X_test_cond, y_train_cond, y_test_cond, idx_train_cond, idx_test_cond = train_test_split(
        X_condition_embeddings,
        y_condition,
        indices_cond,
        test_size=0.2,
        random_state=42,
        stratify=y_condition
    )
else:
    print(f"Warning: Some Condition classes have only 1 sample. Using non-stratified split.")
    print(f"Classes with < 2 samples: {sum(1 for count in class_counts_cond.values() if count < 2)}")
    X_train_cond, X_test_cond, y_train_cond, y_test_cond, idx_train_cond, idx_test_cond = train_test_split(
        X_condition_embeddings,
        y_condition,
        indices_cond,
        test_size=0.2,
        random_state=42
    )

print(f"Condition Training set: {X_train_cond.shape[0]} samples")
print(f"Condition Test set: {X_test_cond.shape[0]} samples")
print(f"Unique Condition classes: {len(np.unique(y_condition))}")


Classes with < 2 samples: 2
Condition Training set: 73609 samples
Condition Test set: 18403 samples
Unique Condition classes: 141


In [ ]:
# Train Problem classifier
print("Training Problem classifier...")
classifier_problem = LogisticRegression(
    max_iter=1000,
    multi_class='multinomial',
    solver='lbfgs',
    C=1.0,
    random_state=42
)
classifier_problem.fit(X_train_prob, y_train_prob)
print("Problem classifier trained!")

# Evaluate on test set
y_pred_prob = classifier_problem.predict(X_test_prob)
accuracy_prob = accuracy_score(y_test_prob, y_pred_prob)
print(f"\nProblem Classification Accuracy: {accuracy_prob:.4f}")
print("\nClassification Report:")
print(classification_report(y_test_prob, y_pred_prob, zero_division=0))


Training Problem classifier...
Problem classifier trained!

Problem Classification Accuracy: 0.7804

Classification Report:
                                               precision    recall  f1-score   support

                AC Control Module Malfunction       0.50      0.33      0.40         3
                       Abnormal Flow/Pressure       0.53      0.61      0.57        49
               Abnormal Operation/Inoperative       1.00      0.04      0.08        48
                              Adhesion Issues       0.80      0.98      0.88       142
                       Adhesive Contamination       0.00      0.00      0.00         1
                         Air Fuel Ratio Issue       0.50      0.02      0.04        48
                  Alignement/Adjustment Issue       0.81      0.83      0.82        42
      Alignment/Calibration/Programming Issue       0.00      0.00      0.00         5
                              Battery Failure       0.86      0.80      0.83        30
     

In [ ]:
# Train Condition classifier
print("Training Condition classifier...")
classifier_condition = LogisticRegression(
    max_iter=1000,
    multi_class='multinomial',
    solver='lbfgs',
    C=1.0,
    random_state=42
)
classifier_condition.fit(X_train_cond, y_train_cond)
print("Condition classifier trained!")

# Evaluate on test set
y_pred_cond = classifier_condition.predict(X_test_cond)
accuracy_cond = accuracy_score(y_test_cond, y_pred_cond)
print(f"\nCondition Classification Accuracy: {accuracy_cond:.4f}")
print("\nClassification Report:")
print(classification_report(y_test_cond, y_pred_cond, zero_division=0))


Training Condition classifier...
Condition classifier trained!

Condition Classification Accuracy: 0.8025

Classification Report:
                                                         precision    recall  f1-score   support

                            AC & Climate Control Issues       0.00      0.00      0.00        10
                               AC & Heating Inoperative       0.63      0.65      0.64        34
                        AC & Temperature Control Issues       0.67      0.15      0.24        27
                                      AC Airflow Issues       0.00      0.00      0.00         4
                                AC Functionality Issues       0.76      0.58      0.66        43
                                 AC Issue/Noise/Leakage       0.00      0.00      0.00        13
                     AC Issue/Not Cooling/Cooling Issue       0.89      0.99      0.94       590
                                  AC System Inoperative       0.85      0.39      0.54       

In [ ]:
# Create constraint-aware prediction function
def predict_with_constraint(classifier, embedding_model, commodity, mode_text, mode_type, 
                           commodity_problem_map, commodity_condition_map):
    """
    Predict Problem or Condition with commodity constraint enforcement
    
    Args:
        classifier: Trained classifier (problem or condition)
        embedding_model: Sentence transformer model
        commodity: Commodity code (can be string or int, will be converted to string)
        mode_text: Failure Mode text (for Problem) or Symptom Mode text (for Condition)
        mode_type: 'problem' or 'condition'
        commodity_problem_map: Dictionary mapping commodity to valid problems
        commodity_condition_map: Dictionary mapping commodity to valid conditions
    """
    # Convert commodity to string to ensure consistent key type
    commodity = str(commodity)
    
    # Create input text
    if mode_type == 'problem':
        input_text = f"Commodity_{commodity} Failure: {mode_text}"
        valid_categories = commodity_problem_map.get(commodity, [])
    else:  # condition
        input_text = f"Commodity_{commodity} Symptom: {mode_text}"
        valid_categories = commodity_condition_map.get(commodity, [])
    
    # Generate embedding
    embedding = embedding_model.encode(
        [input_text],
        convert_to_numpy=True,
        normalize_embeddings=True
    )
    
    # Get prediction probabilities
    probs = classifier.predict_proba(embedding)[0]
    classes = classifier.classes_
    
    # Create probability dictionary
    prob_dict = dict(zip(classes, probs))
    
    # Filter to only valid categories for this commodity
    if valid_categories:
        valid_probs = {cat: prob_dict.get(cat, 0) for cat in valid_categories}
        # Re-normalize
        total_prob = sum(valid_probs.values())
        if total_prob > 0:
            valid_probs = {k: v/total_prob for k, v in valid_probs.items()}
        else:
            # If no valid category has probability, return most likely overall
            return max(prob_dict.items(), key=lambda x: x[1])[0], prob_dict
        # Return category with highest probability
        predicted = max(valid_probs.items(), key=lambda x: x[1])[0]
        return predicted, valid_probs
    else:
        # No constraint available, return regular prediction
        predicted = classifier.predict(embedding)[0]
        return predicted, prob_dict

print("Constraint-aware prediction function created!")


Constraint-aware prediction function created!


In [ ]:
# Evaluate constraint enforcement on test set
print("Evaluating constraint enforcement on test set...\n")

# Problem constraint evaluation - use the test indices from split
test_problem_df = df_problem.iloc[idx_test_prob].copy()

constraint_violations_prob = 0
correct_predictions_prob = 0

for idx, row in test_problem_df.iterrows():
    # Convert commodity to string for consistent dictionary access
    commodity = str(row['Commodity'])
    predicted, _ = predict_with_constraint(
        classifier_problem, embedding_model,
        commodity, row['Failure Mode Text'], 'problem',
        commodity_problem_map, commodity_condition_map
    )
    actual = row['Problem']
    
    # Check if prediction is valid for this commodity
    valid_categories = commodity_problem_map.get(commodity, [])
    if predicted not in valid_categories:
        constraint_violations_prob += 1
    
    if predicted == actual:
        correct_predictions_prob += 1

print(f"Problem Classification:")
print(f"  Constraint violations: {constraint_violations_prob}/{len(test_problem_df)}")
print(f"  Correct predictions: {correct_predictions_prob}/{len(test_problem_df)}")
print(f"  Accuracy with constraints: {correct_predictions_prob/len(test_problem_df):.4f}")

# Condition constraint evaluation - use the test indices from split
test_condition_df = df_condition.iloc[idx_test_cond].copy()

constraint_violations_cond = 0
correct_predictions_cond = 0

for idx, row in test_condition_df.iterrows():
    # Convert commodity to string for consistent dictionary access
    commodity = str(row['Commodity'])
    predicted, _ = predict_with_constraint(
        classifier_condition, embedding_model,
        commodity, row['Symptom Mode Text'], 'condition',
        commodity_problem_map, commodity_condition_map
    )
    actual = row['Condition']
    
    # Check if prediction is valid for this commodity
    valid_categories = commodity_condition_map.get(commodity, [])
    if predicted not in valid_categories:
        constraint_violations_cond += 1
    
    if predicted == actual:
        correct_predictions_cond += 1

print(f"\nCondition Classification:")
print(f"  Constraint violations: {constraint_violations_cond}/{len(test_condition_df)}")
print(f"  Correct predictions: {correct_predictions_cond}/{len(test_condition_df)}")
print(f"  Accuracy with constraints: {correct_predictions_cond/len(test_condition_df):.4f}")


Evaluating constraint enforcement on test set...

Problem Classification:
  Constraint violations: 0/18403
  Correct predictions: 14429/18403
  Accuracy with constraints: 0.7841

Condition Classification:
  Constraint violations: 0/18403
  Correct predictions: 14963/18403
  Accuracy with constraints: 0.8131


In [ ]:
# Create a convenient prediction function for new data
def predict_problem_and_condition(commodity, failure_mode_text, symptom_mode_text):
    """
    Predict both Problem and Condition for given inputs
    
    Args:
        commodity: Commodity code (can be string or int, will be converted to string)
        failure_mode_text: Failure mode text (can be list or string)
        symptom_mode_text: Symptom mode text (can be list or string)
    
    Returns:
        tuple: (predicted_problem, predicted_condition, problem_probs, condition_probs)
    """
    # Convert commodity to string for consistent dictionary access
    commodity = str(commodity)
    
    # Handle list inputs
    if isinstance(failure_mode_text, list):
        failure_mode_text = ' '.join([str(x) for x in failure_mode_text])
    if isinstance(symptom_mode_text, list):
        symptom_mode_text = ' '.join([str(x) for x in symptom_mode_text])
    
    # Predict Problem
    predicted_problem, problem_probs = predict_with_constraint(
        classifier_problem, embedding_model,
        commodity, failure_mode_text, 'problem',
        commodity_problem_map, commodity_condition_map
    )
    
    # Predict Condition
    predicted_condition, condition_probs = predict_with_constraint(
        classifier_condition, embedding_model,
        commodity, symptom_mode_text, 'condition',
        commodity_problem_map, commodity_condition_map
    )
    
    return predicted_problem, predicted_condition, problem_probs, condition_probs

# Example usage
print("Example prediction:")
commodity_example = '87106'  # Use string to match dictionary keys
failure_example = "Stuck - Actuator"
symptom_example = "Noise - Dashboard"

prob, cond, prob_probs, cond_probs = predict_problem_and_condition(
    commodity_example, failure_example, symptom_example
)

print(f"Commodity: {commodity_example}")
print(f"Failure Mode: {failure_example}")
print(f"Symptom Mode: {symptom_example}")
print(f"\nPredicted Problem: {prob}")
print(f"Predicted Condition: {cond}")
print(f"\nTop 3 Problem probabilities:")
for cat, prob_val in sorted(prob_probs.items(), key=lambda x: x[1], reverse=True)[:3]:
    print(f"  {cat}: {prob_val:.4f}")
print(f"\nTop 3 Condition probabilities:")
for cat, prob_val in sorted(cond_probs.items(), key=lambda x: x[1], reverse=True)[:3]:
    print(f"  {cat}: {prob_val:.4f}")


Example prediction:
Commodity: 87106
Failure Mode: Stuck - Actuator
Symptom Mode: Noise - Dashboard

Predicted Problem: Stuck/Seized Components
Predicted Condition: AC/Noise/System Issues

Top 3 Problem probabilities:
  Stuck/Seized Components: 0.5685
  Servo/Motor/Actuator Issue: 0.1586
  Other: 0.1508

Top 3 Condition probabilities:
  AC/Noise/System Issues: 0.8934
  Multiple Lights On/Other Light On/Other Warning Message: 0.0671
  Other: 0.0294


In [ ]:
# Generate predictions for test set and save to CSV
print("Generating predictions for test set...")
print("This may take a few minutes...\n")

# Merge test_problem_df and test_condition_df
# Both should have the same original indices from master dataframe
# Use the index to merge them
test_problem_df_with_idx = test_problem_df.copy()
test_condition_df_with_idx = test_condition_df.copy()

# Merge on index (original dataframe index)
test_combined = test_problem_df_with_idx.merge(
    test_condition_df_with_idx[['Condition', 'Symptom Mode Text']],
    left_index=True,
    right_index=True,
    how='left',
    suffixes=('', '_cond')
)

# If merge didn't work well, try using TWC_No as key
if len(test_combined) == 0 or test_combined['Condition'].isna().all():
    # Try merging on TWC_No if available
    if 'TWC_No' in test_problem_df_with_idx.columns and 'TWC_No' in test_condition_df_with_idx.columns:
        test_combined = test_problem_df_with_idx.merge(
            test_condition_df_with_idx[['TWC_No', 'Condition', 'Symptom Mode Text']],
            on='TWC_No',
            how='left',
            suffixes=('', '_cond')
        )

print(f"Processing {len(test_combined)} test samples...\n")
test_predictions_list = []

# Process each row
for idx, row in test_combined.iterrows():
    commodity = str(row['Commodity'])
    
    # Get Problem prediction
    predicted_problem, problem_probs = predict_with_constraint(
        classifier_problem, embedding_model,
        commodity, row['Failure Mode Text'], 'problem',
        commodity_problem_map, commodity_condition_map
    )
    
    # Get top 3 problem probabilities
    top_problem_probs = sorted(problem_probs.items(), key=lambda x: x[1], reverse=True)[:3]
    problem_confidence = top_problem_probs[0][1] if top_problem_probs else 0.0
    
    # Get Condition prediction if available
    if pd.notna(row.get('Symptom Mode Text', pd.NA)) and str(row.get('Symptom Mode Text', '')) != '':
        symptom_mode_text = row['Symptom Mode Text']
        predicted_condition, condition_probs = predict_with_constraint(
            classifier_condition, embedding_model,
            commodity, symptom_mode_text, 'condition',
            commodity_problem_map, commodity_condition_map
        )
        actual_condition = row.get('Condition', 'N/A')
        
        # Get top 3 condition probabilities
        top_condition_probs = sorted(condition_probs.items(), key=lambda x: x[1], reverse=True)[:3]
        condition_confidence = top_condition_probs[0][1] if top_condition_probs else 0.0
    else:
        predicted_condition = 'N/A'
        actual_condition = 'N/A'
        symptom_mode_text = 'N/A'
        condition_confidence = 0.0
        top_condition_probs = []
    
    # Create record
    record = {
        'TWC_No': row.get('TWC_No', 'N/A'),
        'Commodity': row['Commodity'],
        'Failure_Mode': row['Failure Mode Text'],
        'Symptom_Mode': symptom_mode_text,
        'Actual_Problem': row['Problem'],
        'Predicted_Problem': predicted_problem,
        'Problem_Correct': 1 if predicted_problem == row['Problem'] else 0,
        'Problem_Confidence': problem_confidence,
        'Problem_Top2': top_problem_probs[1][0] if len(top_problem_probs) > 1 else 'N/A',
        'Problem_Top2_Prob': top_problem_probs[1][1] if len(top_problem_probs) > 1 else 0.0,
        'Problem_Top3': top_problem_probs[2][0] if len(top_problem_probs) > 2 else 'N/A',
        'Problem_Top3_Prob': top_problem_probs[2][1] if len(top_problem_probs) > 2 else 0.0,
        'Actual_Condition': actual_condition,
        'Predicted_Condition': predicted_condition,
        'Condition_Correct': 1 if predicted_condition == actual_condition else 0,
        'Condition_Confidence': condition_confidence,
        'Condition_Top2': top_condition_probs[1][0] if len(top_condition_probs) > 1 else 'N/A',
        'Condition_Top2_Prob': top_condition_probs[1][1] if len(top_condition_probs) > 1 else 0.0,
        'Condition_Top3': top_condition_probs[2][0] if len(top_condition_probs) > 2 else 'N/A',
        'Condition_Top3_Prob': top_condition_probs[2][1] if len(top_condition_probs) > 2 else 0.0,
    }
    
    test_predictions_list.append(record)
    
    # Progress indicator
    if len(test_predictions_list) % 1000 == 0:
        print(f"  Processed {len(test_predictions_list)}/{len(test_problem_df)} samples...")

# Create DataFrame
test_predictions_df = pd.DataFrame(test_predictions_list)

print(f"\nGenerated predictions for {len(test_predictions_df)} test samples")
print(f"Problem Accuracy: {test_predictions_df['Problem_Correct'].mean():.4f}")
print(f"Condition Accuracy: {test_predictions_df['Condition_Correct'].mean():.4f}")

# Save to CSV
output_filename = 'test_predictions.csv'
test_predictions_df.to_csv(output_filename, index=False)
print(f"\n✓ Saved test predictions to '{output_filename}'")
print(f"  Total rows: {len(test_predictions_df)}")
print(f"  Total columns: {len(test_predictions_df.columns)}")
print(f"\nColumns: {', '.join(test_predictions_df.columns.tolist())}")
print(f"\nFirst few rows:")
test_predictions_df.head()


Generating predictions for test set...
This may take a few minutes...

Processing 18403 test samples...

  Processed 1000/18403 samples...
  Processed 2000/18403 samples...
  Processed 3000/18403 samples...
  Processed 4000/18403 samples...
  Processed 5000/18403 samples...
  Processed 6000/18403 samples...
  Processed 7000/18403 samples...
  Processed 8000/18403 samples...
  Processed 9000/18403 samples...
  Processed 10000/18403 samples...
  Processed 11000/18403 samples...
  Processed 12000/18403 samples...
  Processed 13000/18403 samples...
  Processed 14000/18403 samples...
  Processed 15000/18403 samples...
  Processed 16000/18403 samples...
  Processed 17000/18403 samples...
  Processed 18000/18403 samples...

Generated predictions for 18403 test samples
Problem Accuracy: 0.7841
Condition Accuracy: 0.8201

✓ Saved test predictions to 'test_predictions.csv'
  Total rows: 18403
  Total columns: 20

Columns: TWC_No, Commodity, Failure_Mode, Symptom_Mode, Actual_Problem, Predicted_P

,TWC_No,Commodity,Failure_Mode,Symptom_Mode,Actual_Problem,Predicted_Problem,Problem_Correct,Problem_Confidence,Problem_Top2,Problem_Top2_Prob,Problem_Top3,Problem_Top3_Prob,Actual_Condition,Predicted_Condition,Condition_Correct,Condition_Confidence,Condition_Top2,Condition_Top2_Prob,Condition_Top3,Condition_Top3_Prob
0,8519842,87106,Failed - Servo Assembly,Noise - Dashboard,Stuck/Seized Components,Servo/Motor/Actuator Issue,0,0.647926,Stuck/Seized Components,0.153003,Electrical/Programming/Communication Issue,0.063795,AC/Noise/System Issues,AC/Noise/System Issues,1,0.893447,Multiple Lights On/Other Light On/Other Warnin...,0.067145,Other,0.029379
1,3690483,88710,Leakage - Rear AC Line,Cooling Issue - Air Conditioning System,Leakage/Poor Machining/Sealing,Leakage/Poor Machining/Sealing,1,0.894547,HVAC Malfunction,0.047803,Coolant & Leakage Issues,0.020027,AC Issue/Not Cooling/Cooling Issue,AC Issue/Not Cooling/Cooling Issue,1,0.989976,Noise/Cooling Issue,0.004117,Leakage/Cooling Issues,0.002767
2,3054183,87106,Operation Issue - AC Servo Motor,Incorrect Mode - Air Outlet,Servo/Motor/Actuator Issue,Servo/Motor/Actuator Issue,1,0.894636,Stuck/Seized Components,0.027343,Open/Short Circuit/Electrical Issue,0.025739,AC Functionality Issues,AC Functionality Issues,1,0.872577,AC Temperature & Airflow Issues,0.042952,Other,0.029255
3,7605513,89904 / 8990H,Binding - Key Fob Button Stuck - Key Fob Button,Activation Failure - Lock Button,Stuck/Seized,Stuck/Seized,1,0.755387,Loose/Fitment/Assembly Issue,0.058350,Key Fob Inoperative/Not Working,0.049614,Lock/Unlock Failure,Button Inoperative/Not Working,0,0.373912,Other,0.146894,Multiple Lights On/Other Light On/Other Warnin...,0.120008
4,6326057,89661,No Condition Mentioned - No Component Mentioned,Turned ON - Check Engine Light Warning Sign - ...,Unclear From CCR Comments,Unclear From CCR Comments,1,0.559687,Programming/Software/Calibration Issues,0.360538,Performance/Timing Issue,0.017706,CEL ON,CEL ON,1,0.848919,Multiple Lights On/Other Light On/Other Warnin...,0.131346,Power/Electrical Issue,0.008427
